## Imports

In [37]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# SciKit-Learn
from sklearn.linear_model import LogisticRegression

# Plotting library
from matplotlib import pyplot
import matplotlib as mpl

# Optimization module in scipy
from scipy import optimize

#Skynet
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing

## Load Data

In [38]:
X_train = np.zeros((7868, 76))
Y_train = np.zeros((7868, 147))
X_test = np.zeros((2705, 76))
predictions = np.zeros((2705, 147))

# Load X_train
dir_string = '../data/Kaggle_Final/train_feature_files'
training_files = os.listdir(dir_string)

for file in training_files:
    file_path = dir_string + '/' + file
    song_id_str = file_path.split('/')[-1].split('.')[0]
    a = np.load(file_path)
    a_summary = a['summary']
    X_train[int(song_id_str)] = a_summary
    
# Load Y_train
with open('../data/Kaggle_Final/cal10k_train_data.csv', 'r') as y_train_file:
    for line in y_train_file:
        if 'id' not in line:
            features_w_id = line.strip().split(',')
            features_no_id = features_w_id[1:]
            for idx in range(len(features_no_id)):
                features_no_id[idx] = int(features_no_id[idx])
            Y_train[int(features_w_id[0])] = np.array(features_no_id)
            
# Load X_test
dir_string = '../data/Kaggle_Final/test_feature_files/'
testing_files = os.listdir(dir_string)

for file in testing_files:
    file_path = dir_string + file
    song_id_str = file_path.split('/')[-1].split('.')[0]
    data = np.load(file_path)
    data_summary = data['summary']
    X_test[int(song_id_str)] = data_summary

## Clean the baby

In [39]:
where_are_NaNs = np.isnan(X_train)
X_train[where_are_NaNs] = 0
where_are_NaNs = np.isnan(X_test)
X_test[where_are_NaNs] = 0
print(X_train.shape)
print(X_test.shape)

(7868, 76)
(2705, 76)


## Process the baby

In [45]:
scaled_X = preprocessing.scale(X_train)
scaled_test = preprocessing.scale(X_test)
print(scaled_X)

[[ 1.04953433e+00 -1.82717419e+00 -3.14378223e-01 ... -7.78459164e-01
  -6.15547973e-01  3.63916714e-02]
 [-5.89573467e-01 -5.43698480e-01  1.26742646e+00 ...  3.02933909e+00
   6.36986019e+00 -3.53863479e-02]
 [-8.71579002e-01 -1.45694081e+00  8.06538601e-01 ... -1.39196830e+00
   7.20935543e-02  7.04298404e-01]
 ...
 [-6.80810552e-01  1.28278619e+00 -6.48520049e-01 ...  1.12104502e+00
   4.24172839e-03 -6.16879755e-01]
 [-8.78688385e-01  4.17609241e-01 -1.75760504e-01 ...  6.52454338e-01
  -1.75135222e-01 -5.08273063e-01]
 [-9.54521806e-01  4.17609241e-01  4.14536114e-01 ...  1.37187461e-01
  -1.69513068e-01 -5.52251472e-01]]


## Fit the baby

In [46]:
genre_models = list()
for i in range(147):
    genre_model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(25, 2), random_state=1)
    genre_model.fit(scaled_X, Y_train[:,i])
    genre_models.append(genre_model)
print(len(genre_models))

147


## Teach the robot how to love

In [47]:
for i in range(147):
    genre_prediction = genre_models[i].predict(X_test)
    genre_prediction_prob = genre_models[i].predict_proba(X_test)
    predictions[:, i] = genre_prediction_prob[:,1]  # Get probability that it is class=1
        
print(predictions)

[[0.97905917 0.         0.         ... 0.         0.         0.        ]
 [0.97905917 0.         0.         ... 0.         0.         0.        ]
 [0.97905917 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.97905917 0.         0.         ... 0.         0.         0.        ]
 [0.97905917 0.         0.         ... 0.         0.         0.        ]
 [0.97905917 0.         0.         ... 0.         0.         0.        ]]


In [48]:
print(np.sum(predictions))

12827.330277677907


## Show the world

In [49]:
file = open('../data/Kaggle_Final/cal10k_test_random_submission.csv', 'r')
id_line = None
for line in file:
    if 'id' in line:
        id_line = line
ids = []
for i in range(2705):
    ids.append(int(i))
predictions = np.insert(predictions, 0, ids, axis=1)
string_predictions = []
for song_idx in range(len(predictions)):
    song = predictions[song_idx]
    song_string = []
    for genre_idx in range(len(song)):
        if genre_idx != 0:
            song_string.append(str(predictions[song_idx, genre_idx]))
        else: 
            song_string.append(str(predictions[song_idx, genre_idx]))
    string_predictions.append(song_string)
with open('../data/Kaggle_Final/skynet_will_rise.csv', 'w') as out_file:
    out_file.write(id_line)
    for song_array in string_predictions:
        song_array[0] = str(int(float(song_array[0])))
        out_file.write(",".join(song_array) + os.linesep)